<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам


In [1]:
# Загрузим все необходимые библиотеки

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import time
import os

from pandas import Series
import re

import json
import requests
import pandas_profiling
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')

from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from datetime import timedelta, datetime, date

from pandas import Series
from bs4 import BeautifulSoup

import seaborn as sn
%matplotlib inline
from sklearn.metrics import accuracy_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve

from datetime import date
from datetime import datetime, timedelta

from sklearn.preprocessing import PolynomialFeatures



In [2]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

Сначала мы создаём список из доступных ссылок, что было проделано в google collab, в надежде,что найдутся сокомандники) (так и не нашлись, к сожалению) После этого мы прошлись по всем найденым ссылкам для получения необходимых данных, проделано в jupyter notebook. 

In [6]:
"""# загружаем необходимые библиотеки для получения и обработки данных

    import requests
    from bs4 import BeautifulSoup
    import csv
    import time
    import json
    import re
    import pandas as pd
   
    from google.colab import drive"""
  
    
"""drive.mount('/drive')


def get_all_links(url_list):
    
    #Функция для получения и сохранения в отдельный файл ссылок на объявления
    по машинам выбранных марок. В результате выполнения получаем файл со 
    списком найденных ссылок.


    all_car = []
    # проходимся по каждой выбранной марке
    # we go through each selected brand
    for url in url_list:
        # получаем количество страниц по выбранной марке
        # get the number of pages for the selected brand
        print(url)
        res = requests.get(url)
        soup = BeautifulSoup(res.content.decode('utf-8'), 'html.parser')
        news_blocks = soup.find('button',
                                class_='Button Button_color_blue Button_size_m Button_type_button Button_width_full')\
            .find('span', class_='ButtonWithLoader__content').text
        page_total = news_blocks.replace(u'\xa0', u' ').split()[1:3]
        page_total = int(''.join(page_total))
        # 38 ads on one page, get the number of pages
        number_of_pages = int(page_total / 38)

        # получаем и сохраняем ссылки по каждой машине из выбранной марки
        # get and save links for each car from the selected brand
        for i in range(1, number_of_pages):
            print('Current Page:', i, 'Total Pages:', number_of_pages)
            url_page = url + '/?page=' + str(i)
            res = requests.get(url)
            # задержка времени при отправке запросов/time delay when sending requests
            time.sleep(0.1)
            soup = BeautifulSoup(res.content.decode('utf-8'), 'html.parser')
            car_page = soup.find_all('a', class_='Link ListingItemTitle__link')
            car_list = list(map(lambda x: x.get('href'), car_page))
            for row in car_list:
                all_car.append(row)
    # записываем новую ссылку на объявление в файл
    # write the new link to the ad to a file
    with open('/drive/My Drive/all_car_27082021.csv', 'a') as myfile:
        wr = csv.writer(myfile)
        for row in all_car:
            wr.writerow([row])
    return all_car


# определяем список марок машин, которые нам интересны
# we define a list of car brands that are of interest to us
brands = ['honda', 'skoda', 'audi', 'volvo', 'bmw', 'nissan', 'infiniti',
          'mercedes', 'toyota', 'lexus', 'volkswagen', 'mitsubishi']
#url_list = ['https://auto.ru/moskva/cars/'+brand+'/all' for brand in brands]

url_list = ['https://auto.ru/cars/'+brand+'/all' for brand in brands]
all_car = get_all_links(url_list)
print(f'Количество отобранных объявлений: {len(all_car)}')"""



"131478 rows"

"df.to_csv('/drive/My Drive/lit_of_links.csv')"
 
    
    
"""# загружаем необходимые библиотеки для получения и обработки данных
# load the necessary libraries for receiving and processing data
import requests
from bs4 import BeautifulSoup
import csv
import time
import json
import re
import pandas as pd"""

"autoru_gdpr=1"

"""def get_cars_info(links, step=0):

    cars_info = pd.DataFrame()
    
    for link in links:
        try:
            response = requests.get(link)
            page = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
            print(f'link={link}, {response.status_code}')
        except:
            print(f'Error get link={link}')
            continue  
        try:
            json_data = json.loads(
                page.find('script', type="application/ld+json").string)
        except:
            json_data = None
        try:
            catalog_url = page.find(
                'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default').get('href')
            response_catalog = requests.get(catalog_url)
            page_catalog = BeautifulSoup(
                response_catalog.content.decode('utf-8'), 'html.parser')
        except:
            print(f'Error get link={link}')
            continue  
        try:
            json_data_catalog = json.loads(page_catalog.find(
                'script', type="application/json", id='initial-state').string)
        except:
            json_data_catalog = None
        try:
            json_data_equip = json.loads(
                page.find('script', type="application/json", id='initial-state').string)
        except:
            json_data_equip = None
        try:
            json_data['description'] = json_data['description'].replace(
                '\n', ' ')
            json_data['description'] = re.sub(
                '\W+', ' ', json_data['description'])
        except:
            json_data['description'] = ''
        try:
            options_dict = json_data_catalog['state']['compare']['selected'][0]['options']
            options_list = [key for key in list(
                options_dict.keys()) if options_dict[key] == 1]
        except:
            options_list = []
      
        try:
            car_url = json_data['offers']['url']
        except:
            car_url = None
        try:
            bodyType = json_data['bodyType']
        except:
            bodyType = None
        try:
            brand = json_data['brand']
        except:
            brand = None
        try:
            color = json_data['color']
        except:
            color = None
        try:
            complectation_dict = options_list
        except:
            complectation_dict = None
        try:
            description = json_data['description']
        except:
            description = None
        try:
            engineDisplacement = json_data['vehicleEngine']['engineDisplacement']
        except:
            engineDisplacement = None
        try:
            enginePower = json_data['vehicleEngine']['enginePower']
        except:
            enginePower = None
        try:
            equipment_dict = json_data_equip['card']['vehicle_info']['equipment']
        except:
            equipment_dict = None
        try:
            fuelType = json_data['fuelType']
        except:
            fuelType = None
        try:
            image = json_data['image']
        except:
            image = None
        try:
            mileage = page.find('li', class_='CardInfoRow CardInfoRow_kmAge').find_all(
                'span')[-1].text.replace(u'\xa0', u' ')
        except:
            mileage = None
        try:
            modelDate = json_data['modelDate']
        except:
            modelDate = None
        try:
            model_name = page.find_all(
                'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
        except:
            model_name = None
        try:
            name = json_data['name']
        except:
            name = None
        try:
            numberOfDoors = json_data['numberOfDoors']
        except:
            numberOfDoors = None
        try:
            parsing_unixtime = int(time.time())
        except:
            parsing_unixtime = None
        try:
            price = json_data['offers']['price']
        except:
            price = None
        try:
            priceCurrency = json_data['offers']['priceCurrency']
        except:
            priceCurrency = None
        try:
            productionDate = json_data['productionDate']
        except:
            productionDate = None
        try:
            sell_id = page.find(
                'div', class_='CardHead__infoItem CardHead__id').text[2:]
        except:
            sell_id = None
        try:
            views = page.find(
                'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
        except:
            views = None
        try:
            date_added = page.find(
                'div', class_='CardHead__infoItem CardHead__creationDate').text
        except:
            date_added = None
        try:
            super_gen = json.loads(
                page.find('div', id="sale-data-attributes").get('data-bem'))
        except:
            super_gen = None
        try:
            vehicleConfiguration = json_data['vehicleConfiguration']
        except:
            vehicleConfiguration = None
        try:
            vehicleTransmission = json_data['vehicleTransmission']
        except:
            vehicleTransmission = None
        try:
            Владельцы = page.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all(
                'span')[-1].text.replace(u'\xa0', u' ')
        except:
            Владельцы = None
        try:
            Владение = page.find(
                'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
        except:
            Владение = None
        try:
            ПТС = page.find('li', class_='CardInfoRow CardInfoRow_pts').find_all(
                'span')[-1].text
        except:
            ПТС = None
        try:
            Привод = page.find('li', class_='CardInfoRow CardInfoRow_drive').find_all(
                'span')[-1].text
        except:
            Привод = None
        try:
            Руль = page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all(
                'span')[-1].text
        except:
            Руль = None
        try:
            Состояние = page.find(
                'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
        except:
            Состояние = None
        try:
            Таможня = page.find(
                'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
        except:
            Таможня = None
        try:
            region = page.find('div', class_='CardBreadcrumbs').find_all(
                'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
        except:
            region = None

        row = {'car_url': car_url, 'bodyType': bodyType, 'brand': brand, 'color': color,
               'complectation_dict': complectation_dict, 'description': description,
               'engineDisplacement': engineDisplacement, 'enginePower': enginePower,
               'equipment_dict': equipment_dict, 'fuelType': fuelType, 'image': image,
               'mileage': mileage, 'modelDate': modelDate, 'model_name': model_name,
               'name': name, 'numberOfDoors': numberOfDoors, 'parsing_unixtime': parsing_unixtime,
               'price': price, 'priceCurrency': priceCurrency, 'productionDate': productionDate,
               'sell_id': sell_id, 'views': views, 'date_added': date_added,
               'super_gen': super_gen, 'vehicleConfiguration': vehicleConfiguration,
               'vehicleTransmission': vehicleTransmission, 'Владельцы': Владельцы,
               'Владение': Владение, 'ПТС': ПТС, 'Привод': Привод, 'Руль': Руль,
               'Состояние': Состояние, 'Таможня': Таможня, 'region': region}
        
        cars_info = pd.concat([cars_info, pd.DataFrame([row])])
        step += 1
        if step % 10 == 0:
            print(f'Completed {step} of total {len(links)}')
        else:
            continue
    return cars_info

df = pd.read_csv('lit_of_links.csv', sep=',', header=None)
all_links = df[1].tolist()"""


"""# проходимся по каждому объявлению из всего списка или выбранного диапозона,
# собираем данные по автомобилю и записываем их в файл
# we go through each ad from the entire list or the selected range,
# collect data on the car and write it to a file
car_data_1 = get_cars_info(all_links[:50000])
car_data_1.reset_index()
car_data_1.to_csv('all_car_1.csv')
car_data_2 = get_cars_info(all_links[50000:100000])
car_data_2.reset_index()
car_data_2.to_csv('all_car_2.csv')
car_data_3 = get_cars_info(all_links[100000:])
car_data_3.reset_index()
car_data_3.to_csv('all_car_3.csv')

df_1=pd.read_csv("all_car_1.csv")
df_2=pd.read_csv("all_car_2.csv")
df_3=pd.read_csv("all_car_3.csv")
all_car=pd.concat([df_1,df_2,df_3], sort=False, axis=0)
all_car.to_csv('all_cars.csv')"""

# Setup

Изначально проводилось обучение только на спарсенных лично данных, но по какой-то причине в датасете было очень много дубликатов, скорее всего ошибка в самом парсинге. Поэтому используем также и датасет данный в baseline

In [7]:
VERSION    = 8
DIR_TRAIN  = '../input/all-cars/' # подключил к ноутбуку внешний датасет
DIR_TRAIN_base  = '../input/parsing-all-moscow-auto-ru-09-09-2020/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [8]:
!ls '../input'

In [9]:
train = pd.read_csv(DIR_TRAIN+'all_cars.csv') # датасет для обучения модели
train_base  = pd.read_csv(DIR_TRAIN_base+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [10]:
train.head(5)

Убираем ненужные колонки

In [11]:
train = train.drop('Unnamed: 0', 1)

In [12]:
train = train.drop('Unnamed: 0.1', 1)

In [13]:
train.info()

In [14]:
# Смотрим совпадения по всем колонкам кроме patsing_unixtime

train.duplicated(subset=['car_url', 'bodyType', 'brand', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_name', 'name',
       'numberOfDoors', 'price', 'priceCurrency',
       'productionDate', 'sell_id', 'views', 'date_added', 'super_gen',
       'vehicleConfiguration', 'vehicleTransmission', 'Владельцы', 'Владение',
       'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня', 'region']).sum()

In [15]:
# Удаляем дубликаты

train.drop_duplicates(subset=['car_url', 'bodyType', 'brand', 'color', 'complectation_dict', 'description', 'engineDisplacement',
        'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_name', 'name', 'numberOfDoors', 'price',
        'priceCurrency', 'productionDate', 'sell_id', 'views', 'date_added', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission',
        'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня', 'region'], keep = 'first', inplace = True)

In [16]:
train

In [17]:
train_base.head(5)

In [18]:
train_base.info(5)

In [19]:
train_base.columns

In [20]:
train_base.drop_duplicates(subset=['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'], keep = 'first', inplace = True)

In [21]:
train_base

In [22]:
test.info()

In [23]:
test.sample(10)

In [24]:
test.duplicated(subset=['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня']).sum()

In [25]:
# определяем, какие столбцы не вошли в датасет
drop_columns = []
for i in train_base.columns:
    if i not in train.columns:
        drop_columns.append(i)

In [26]:
drop_columns_2 = []
for i in train.columns:
    if i not in train_base.columns:
        drop_columns_2.append(i)

In [27]:
drop_columns_3 = []
for i in test.columns:
    if i not in train.columns:
        drop_columns_3.append(i)

In [28]:
drop_columns_4 = []
for i in train.columns:
    if i not in test.columns:
        drop_columns_4.append(i)

In [29]:
drop_columns_5 = []
for i in test.columns:
    if i not in train_base.columns:
        drop_columns_5.append(i)

In [30]:
drop_columns_6 = []
for i in train_base.columns:
    if i not in train.columns:
        drop_columns_6.append(i)

In [31]:
train_base.columns

In [32]:
train.columns

In [33]:
test.columns

In [34]:
# активируем функцию
drop_columns

In [35]:
drop_columns_2

In [36]:
drop_columns_3

In [37]:
drop_columns_4

In [38]:
drop_columns_5

In [39]:
drop_columns_6

In [40]:
train_base.Комплектация.sample(10)

In [41]:
train_base['Комплектация'].iloc[7318]

Здесь много информации и той, которая есть в других колонках и уникальная, в дальнейшим возможно стоит поработать с этим признаком, но на данный момент удаляем

In [42]:
train_base = train_base.drop('Комплектация', 1) 

In [43]:
train_base.start_date.sample(3)

Тоже самое, что и date_added в test

In [44]:
train['date_added'].sample(3)

In [45]:
train_base.hidden.sample(3)

полностью пустой признак, удаляем

In [46]:
train_base = train_base.drop('hidden', 1) 

In [47]:
train_base.model.sample(3)

тоже самое, что и model_name в test и train, надо переименовать

In [48]:
train_base = train_base.rename(columns={'model': 'model_name'})

In [49]:
train.complectation_dict.sample(10)

In [50]:
train.equipment_dict.sample(10)

In [51]:
train.super_gen.sample(10)

In [52]:
test['vendor'].sample(10)

Данную информацию мы не собрали при парсинге, при этом логически, марки машин отчасти показывают страну производства, поэтому пока можем удалить этот столбец. В дальнейшим, можем добавить информацию, анализирую марки машин

In [53]:
test = test.drop('vendor', 1) 

In [54]:
test['model_info'].sample(10)

Данная информация есть в других ячейках, можем удалить. В дальнейшим мы увидем, что есть пропуски по моделям машин, опять же можно было бы проверить наличие данной информации в данном столбце, как будет больше времени, надо будет вернуться к этому

In [55]:
test = test.drop('model_info', 1)

Переменные views, region и date_added могут быть полезны, если в дальнейшим тестовая выборка пополнится данными признаками, но пока удаляем их

In [56]:
train['views'].sample(10)

In [57]:
train = train.drop('views', 1)

In [58]:
train = train.drop('date_added', 1)

In [59]:
train_base = train_base.drop('start_date', 1)

In [60]:
train['region'].sample(10)

In [61]:
train = train.drop('region', 1)

In [62]:
train_base.info()

## Price

In [63]:
train['price'].value_counts()

In [64]:
train['price'].isnull().sum()

Так как цена наш целевой признак, мы удаляем строки, в которых нет цены

In [65]:
train.dropna(subset=['price'], inplace=True)

In [66]:
train.info()

In [67]:
train

In [68]:
train.price.describe()

In [69]:
train['price'].sort_values()

In [70]:
train_base['price'].value_counts()

In [71]:
train_base['price'].isnull().sum()

In [72]:
train_base.dropna(subset=['price'], inplace=True)

In [73]:
train_base.price.describe()

In [74]:
train_base['price'].sort_values()

## Data Preprocessing


In [75]:
# список числовых переменных
num_cols = []
# список категориальных переменных
cat_cols = []
# список бинарных переменных
bin_cols = []

In [76]:
train['sample'] = 1 # помечаем, где у нас трейн
train_base['sample'] = 1 # помечаем, где у нас трейн
test['sample'] = 0 # помечаем, где у нас тест
test['price'] = 0 # значение целевой переменной отсутствует для тестовой выборки
data = test.append(train, sort=False).reset_index(drop=True)

In [77]:
data.head(5)

Я сначала соединила test и train, чтобы проверить, есть ли дубликаты по sell_id. Так как в train_base такого значения нет, то при объединение они одинаково заполнятся na

In [78]:
data.duplicated(subset=['sell_id']).sum()

In [79]:
# Оставляем первые дубликаты, так как они тестовые
data.drop_duplicates(subset=['sell_id'], keep = 'first', inplace = True)

In [80]:
data=data.append(train_base, sort=False).reset_index(drop=True)

In [81]:
#Приведём колонки к одному виду названий
data.rename(columns={'bodyType': 'body_type',
                   'engineDisplacement': 'engine_displacement',
                   'enginePower': 'engine_power',
                   'fuelType': 'fuel_type',
                   'modelDate': 'model_date',
                   'numberOfDoors': 'number_of_doors',
                   'productionDate': 'production_date',
                   'vehicleConfiguration': 'vehicle_configuration',
                   'vehicleTransmission': 'vehicle_transmission',
                   'Владельцы': 'owners',
                   'Владение': 'ownership_time',
                   'ПТС': 'licence',
                   'Привод': 'type_of_drive',
                   'Руль': 'steering_wheel',
                    'Таможня': 'сustoms',
                    'Состояние': 'condition'
                    }, inplace=True)

In [82]:
data.info()

In [83]:
data.sample(10)

### Pandas Profiling
Производим быстрый и углубленный EDA ещё не очищенных данных.

In [84]:
pandas_profiling.ProfileReport(data)

1) В датасете представлено 30 признака (без sample и price), из них:

* категориальных переменных - 19,
* числовых переменных - 6,
* ссылки - 2,
* bool - 2,
* неподдерживаемых признаков - 4.

2) Больше всего пропусков в sell_id (70.6%), что логично, так как во втором трейне нет этой информации, и ownership time (66.7%), имеет смысл их удалить

3) Также сожем удалить константный признак priceCurrency, condition (+ в нём много прпусков)

Далее будем испольлзовать данные предоставленные pandas_profiling при обработки признакаов

In [85]:
data.info()

In [86]:
data = data.drop('priceCurrency', 1)

In [87]:
data = data.drop('ownership_time', 1)

In [88]:
data = data.drop('sell_id', 1)

In [89]:
data = data.drop('condition', 1)

car_url, parsing_unixtime удаляем, они нам не нужны

In [90]:
data = data.drop('car_url', 1)

In [91]:
data = data.drop('parsing_unixtime', 1)

vehicle_configuration - вся информация у нас есть в других столбцах удаляем

In [92]:
data = data.drop('vehicle_configuration', 1)

complectation_dict, equipment_dict, super_gen - это словари данных, сложны в обработке + с большим количеством пропусков, на данный момент удаляем. При наличие дополнительного времени стоит постараться вытащить дополнительную информацию

In [93]:
data = data.drop('complectation_dict', 1)

In [94]:
data = data.drop('equipment_dict', 1)

In [95]:
data = data.drop('super_gen', 1)

In [96]:
data.info()

Посмотрим каждый признак

### Body_type

Пропущенное значение 1, 192 варианта типа кузова. Посмотрим их

In [97]:
data.body_type.value_counts()

In [98]:
data[data.body_type.isnull() == True]

В этой строке очень мало ифнормации, удалим её

In [99]:
data.dropna(subset=['body_type'], inplace=True) 

In [100]:
data.body_type.isnull().sum()

Можем убрать данные по количеству дверей (не теряя информации, так как в столбце количества дверей ,был всего один пропуск в той же строке, что и у body_type), что уменьшит количество вариаций. 

In [101]:
data['body_type'] = data['body_type'].astype(str).apply(lambda x: None if x.strip()=='' else x)
data['body_type'] = data.body_type.apply(lambda x: x.split(' ')[0].lower())

In [102]:
data.body_type.value_counts()

In [103]:
#определяем его в "категориальные"
cat_cols.append('body_type')

### Brand

Пропусков нет

In [104]:
data.brand.value_counts()

In [105]:
cat_cols.append('brand')

### Color

Пропусков нет

In [106]:
data.color.value_counts()

In [107]:
#Приведём цвета к одному знаминателю
color_dict = {'040001':'black', 
              'FAFBFB':'white',
              'CACECB':'silver',
              '97948F':'grey',
              'чёрный':'black',
              '0000CC':'blue',
              'белый':'white',
              '200204':'brown',
              'EE1D19':'red',
              'серебристый':'silver',
              'серый':'grey',
              'синий':'blue',
              '007F00':'green',
              'C49648':'beige',
              'красный':'red', 
              'коричневый':'brown',
              '22A0F8':'light_blue',
              'зелёный':'green',
              '660099':'purple',
              'DEA522':'gold',
              '4A2197':'violet',
              'бежевый':'beige',
              'FFD600':'yellow',
              'голубой':'light_blue',
              'FF8649':'orange',
              'золотистый':'gold',
              'пурпурный':'purple',
              'фиолетовый':'violet',
              'жёлтый':'yellow',
              'оранжевый':'orange',
              'FFC0CB':'pink',
              'розовый':'pink'
             }
data['color'] = data['color'].map(color_dict)

In [108]:
data.color.value_counts()

In [109]:
cat_cols.append('color')

### Description

Если дополнительно поработать с данным признаком, можно также посмотреть уникальные слова, но на данный момент посмотрим количество символов

In [110]:
data['description']=data['description'].fillna(0)

In [111]:
# сгенерируем числовой признак - количество символов в описании
data['description_len'] = data.description.apply(lambda x: len(str(x)))
# сам description удалим
data = data.drop('description',axis=1)

In [112]:
#Определим его как числовой признак
num_cols.append('description_len')

### Engine_displacement

In [113]:
data['engine_displacement'].value_counts()

In [114]:
data['engine_displacement'].unique()

In [115]:
# Смотрим машины у которых литраж обозначен просто как LTR
data[data['engine_displacement'] == ' LTR']

In [116]:
data[data['engine_displacement']==' LTR'].fuel_type.value_counts()

Машины все электрические, поэтому у них просто нет литража, можем заменить на 0

In [117]:
data['engine_displacement'] = data['engine_displacement'].apply(lambda x: '0 LTR' if x == 'LTR' or x==' LTR' else x)

In [118]:
# оставим только числовые данные.
data['engine_displacement'] = data['engine_displacement'].astype(str)
data['engine_displacement'] = data['engine_displacement'].str.extract(
    '(\d.\d)', expand=False).str.strip()
# Заменим все некорректные числа(объём двигателя в литрах) более 10 на None, при наличие времени, можно точечено найти информацию про литражи
data['engine_displacement'] = data['engine_displacement'].apply(
    lambda x: None if float(x) > 10 else x).astype(float)

In [119]:
data['engine_displacement'].unique()

In [120]:
data['engine_displacement'].value_counts()

In [121]:
data['engine_displacement'].isnull().sum()

In [122]:
data['engine_displacement']=data['engine_displacement'].fillna("2.0")

In [123]:
# Приведем значение признака  в float
data['engine_displacement'] = data['engine_displacement'].astype('float64')

In [124]:
num_cols.append('engine_displacement')

### Engine_power

1 пропуск, и pandasprofiling не может прочитать этот признак

In [125]:
data.engine_power.value_counts()

In [126]:
# Преобразовываем значения столбца, избавляемся от N12
data['engine_power'] = data['engine_power'].astype(str)
data['engine_power'] = data['engine_power'].apply(lambda x: float(x.split(" ")[0]))

In [127]:
data.engine_power.value_counts()

In [128]:
data.engine_power.unique()

In [129]:
# Приведем значение признака  в int
data['engine_power'] = data['engine_power'].astype('int64')

In [130]:
num_cols.append('engine_power')

### Fuel_type

Пропущенных значений нет, 6 уникальных значений

In [131]:
data.fuel_type.value_counts()

In [132]:
cat_cols.append('fuel_type')

### Image

Можно посмотреть влияет ли уникальность(реальность) фотографии на целевой признак

In [133]:
data['image'].isnull().sum()

In [134]:
# Посмотрим на запись
display(data[data.image.isnull() == True])

Так как отсутсвие изображения можно расценивать как отсутствие оригинальной фотографии, то заполняем пропуски самым встречающимся значением

In [135]:
data.image.value_counts()

In [136]:
data['image']=data['image'].fillna(data['image'].mode()[0])

In [137]:
data['image'].isnull().sum()

In [138]:
data['real_photo'] = data.image.apply(lambda x: x.find('get-verba') == -1)

In [139]:
data['real_photo'].value_counts()

In [140]:
data = data.drop('image',axis=1)

In [141]:
bin_cols.append('real_photo')

### Mileage

Pandas Profiling не смог дать нам информацию по данному признаку, будем смотреть почему

In [142]:
data.mileage.isnull().sum()

In [143]:
# Проверим километраж относительно даты выпуска машины
data[data.mileage.isnull()== True].production_date.value_counts()

Судя по всему это новые машины, поэтому пробега нет, заполняем пропуски нулями

In [144]:
data.mileage=data.mileage.fillna("0")

In [145]:
data.mileage.value_counts()

In [146]:
data.mileage.sample()

При попытке перевести в числовой формат на данном этапе появилась ошибка, что есть буквы км. При просмотре sample можно увидеть данные значения. Уберём лишнее

In [147]:
data['mileage'] = data['mileage'].apply(lambda x: str(x).replace('км', '').replace('\xa0', ' '))

In [148]:
data['mileage'].unique()

In [149]:
# Убираем оставшиеся пробелы
data['mileage'] = data['mileage'].apply(lambda x: str(x).replace(' ', ''))

In [150]:
data.mileage.value_counts()

In [151]:
# Преобразуем mileage в int
data['mileage'] = data['mileage'].astype('int64')

In [152]:
num_cols.append('mileage')

### Model_date

1 пропуск, мы его убрали ранее, 79 уникальных значений. Сильная корреляция с productionDate. В дальнейшим можем рассмотерть данные признаки для создания новых

In [153]:
data.model_date.sort_values()

In [154]:
data['model_date'].hist(figsize=(5,5), bins=50)

У нас достаточно сильный перекос моделей на те, которые созданы после 2000. Но те модели, которые были выпущены ранее, могут являтся колекционными и мы их должны оставить, чтобы модель могла предсказывать их стоимость

In [155]:
num_cols.append('model_date')

### Model_name

Много уникальных значений - 705

In [156]:
data.model_name.value_counts()

Заполним пробелы noDate, возможно в одном из признаков, которых мы удалили ранее, есть информация про модель. В дальнейшим, можно будем проверить

In [157]:
data.model_name = data.model_name.fillna('noDate')

In [158]:
cat_cols.append('model_name')

### Name

Судя по данным, Name это смесь признаков, которые у нас уже есть. Поверим ещё раз и удалим, если это так. Можно было бы дозаполнить пропуски в model_name, где возможно, но это весьма долгая и кропотливая работа, в будущем стоит попробовать

In [159]:
data.name.value_counts()

In [160]:
data = data.drop('name',axis=1)

### Number_of_doors

In [161]:
data.number_of_doors.value_counts()

Видим, что есть 2 машины без дверей, посмотрим в каких строках они находятся

In [162]:
data.number_of_doors.sort_values()

In [163]:
data.iloc[16944]

In [164]:
data.iloc[121800]

У данных модели действительно нет дверей

In [165]:
cat_cols.append('number_of_doors')

### Production_date

Нет пустых значений, уникальных значений 81. Сильная корреляция с Model_date. В дальнейшим можем рассмотерть данные признаки для создания новых

In [166]:
data.production_date.value_counts()

In [167]:
data['production_date'].hist(figsize=(5,5), bins=50)

Тоже самое, что и с model_date, сильный перекос машин в сторону произведённых после 2000. Но те машины, которые были выпущены ранее, могут являтся колекционными и мы их должны оставить, чтобы модель могла предсказывать их стоимость

In [168]:
num_cols.append('production_date')

### Vehicle_transmission

In [169]:
data.vehicle_transmission.value_counts()

In [170]:
change_transmission = {
               'автоматическая': 'AUTOMATIC',
               'механическая': 'MECHANICAL',
               'вариатор': 'VARIATOR',
               'роботизированная': 'ROBOT'
                }
data = data.replace({'vehicle_transmission': change_transmission})

In [171]:
data.vehicle_transmission.value_counts()

In [172]:
cat_cols.append('vehicle_transmission')

### Owners

In [173]:
data['owners'].value_counts()

In [174]:
data['owners'].isnull().sum()

In [175]:
# Посмотрим количество владельцов относительно года выпуск
data[data.owners.isnull()== True].production_date.value_counts()

Скорее всего так как они достаточно новые, это может быть либо 1 владелец, либо вообще ноль. Заменим на ноль, чтобы всё-таки иметь отдельную категорию тех машин, у которых не было ещё владельцев

In [176]:
data.owners = data.owners.fillna('0')

In [177]:
data['owners'].unique()

In [178]:
# Создаем словарь с кодировкой значений в числовые
owners_dict = {"3 или более": 3, "1\xa0владелец": 1, "1 владелец": 1, "2\xa0владельца": 2, "2 владельца": 2}
data['owners'].replace(to_replace=owners_dict, inplace=True)  # заменяем значения в соответствии со словарем

In [179]:
# Преобразуем owners в int
data['owners'] = data['owners'].astype('int64')

In [180]:
data['owners'].value_counts()

In [181]:
cat_cols.append('owners')

### Licence

Всего 4 уникальных значения и 591 пропуск

In [182]:
data.licence.value_counts()

In [183]:
change_passport = {
               'Оригинал': 'ORIGINAL',
               'Дубликат': 'DUPLICATE'
                }
data = data.replace({'licence': change_passport})


In [184]:
data.licence.isnull().sum()

In [185]:
data.licence = data.licence.fillna('ORIGINAL')# заполним пропуск наиболее частым значением

In [186]:
# Преобразуем значения в колонке licence
data['licence'] = data['licence'].apply(lambda x: 1 if x == 'ORIGINAL' else 0).astype('int8')

In [187]:
#определяем в бинарные признаки
bin_cols.append('licence')

### Type_of_drive

3 уникальных значения, 234 пропуска

In [188]:
data.type_of_drive.value_counts()

In [189]:
data.type_of_drive = data.type_of_drive.fillna('передний')# заполним пропуск наиболее частым значением, но можно будет посмотреть удаленные признаки, возможно там бдет эта информация

In [190]:
#определяем его в "категориальные"
cat_cols.append('type_of_drive')

### Steering_wheel

4 уникальных значения, 233 пропуска

In [191]:
data.steering_wheel.value_counts()

In [192]:
change_wheel = {
               'Левый': 'LEFT',
               'Правый': 'RIGHT'
                }
data = data.replace({'steering_wheel': change_wheel})

In [193]:
data.steering_wheel = data.steering_wheel.fillna('LEFT')# заполним пропуск наиболее частым значением

In [194]:
# Преобразуем значения в колонке
data['steering_wheel'] = data['steering_wheel'].apply(lambda x: 1 if x == 'LEFT' else 0).astype('int8')

In [195]:
data.steering_wheel.value_counts()

In [196]:
#определяем в бинарные признаки
bin_cols.append('steering_wheel')

## Customs

In [197]:
data.сustoms.value_counts()

In [198]:
data.сustoms.isnull().sum()

Так как значение всего одно, можем удалить признак

In [199]:
data = data.drop('сustoms',axis=1)

## Feature Engineering

In [200]:
#создадим новые признаки на основе существующих: количество лет от производства и возраст модели
data['car_age'] = 2021 - data.production_date
data['model_age'] = 2021 - data.model_date

In [201]:
#определим эти признаки в числовые
num_cols.append('car_age')
num_cols.append('model_age')

## EDA

In [202]:
sn.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sn.heatmap(data.corr(), square=True,
              annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")

In [203]:
# Посмотрим матрицу коэффициента корреляции численных признаков
sn.set(font_scale = 1)
fig, axis = plt.subplots(figsize=(7,7))
sn.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1, square=True,
           annot=True, fmt=".2f", linewidths=0.1)

Помимо очевидной кореляции между production_date, model_date,car_age и model_age. Также с данными показателями скорелирован mileage

Посмотрим распределение целевой переменной числовых признаков

In [204]:
imp_num = pd.Series(f_classif(data[num_cols], data['price'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

description_len самый значимый признак из числовых, model_age самый важный из 4 скореллированных признаков, удалим 3 других

In [205]:
num_cols.remove('car_age')
num_cols.remove('production_date')
num_cols.remove('model_date')

In [206]:
data = data.drop('production_date',axis=1)
data = data.drop('car_age',axis=1)
data = data.drop('model_date',axis=1)

In [207]:
data.info()

In [208]:
bin_cols

In [209]:
cat_cols

In [210]:
num_cols

In [211]:
# трансформируем бинарные признаки

label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
    
data.sample(10)

In [212]:
imp_bin = pd.Series(f_classif(data[bin_cols], data['price'])[0], index = bin_cols)
imp_bin.sort_values(inplace = True)
imp_bin.plot(kind = 'barh')

Наличие фотографий действительно сильно взаимосвязан с ценой

In [213]:
# Посмотрим матрицу коэффициента корреляции бинарных признаков
sn.set(font_scale = 1)
fig, axis = plt.subplots(figsize=(7,7))
sn.heatmap(data[bin_cols].corr().abs(), vmin=0, vmax=1, square=True,
           annot=True, fmt=".2f", linewidths=0.1)

Признаки не скореллированы

In [214]:
for colum in cat_cols:
    data[colum] = data[colum].astype('category').cat.codes

In [215]:
imp_cat = pd.Series(f_classif(data[cat_cols], data['price'])[0], index = cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Количество владельце самый значимый признак из категориальных

## Label Encoding

In [216]:
data

In [217]:
# Преобразуем категориальные признаки
data = pd.get_dummies(data, columns=cat_cols)

In [218]:
# выполним стандартизацию численных переменных
#data[num_cols] = pd.DataFrame(StandardScaler().fit_transform(data[num_cols]), columns=num_cols)

Результаты от стандартизации хуже

In [219]:
data

In [220]:
data.info()

## Train Split

In [221]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample','price'], axis=1)
X = train_data.drop(['price'], axis=1)
y = train_data['price']
y = y/1.3 #согласно автостату цены увеличились на 30%, значит уменьшим нашу целевую переменную
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [222]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train


In [223]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engine_displacement'].map(tmp_train.groupby('engine_displacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


# Fit

In [224]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [225]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

In [226]:
# попробуем подобрать параметры 
cb = CatBoostRegressor(iterations = 20000,
                       learning_rate = 0.15,
                       depth = 9,
                       random_seed = RANDOM_SEED,
                       eval_metric = 'MAPE',
                       custom_metric = ['R2', 'MAE'],
                       silent=True
                       )
cb.fit(X_train, y_train,
         eval_set = (X_test, y_test),
         verbose_eval = 0,
         use_best_model = True
         )

cb.save_model('catboost_single_model_baseline.cb')
predict = cb.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    


In [227]:
model_log = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model_log.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         )

model_log.save_model('catboost_single_model_2_baseline.model')
predict_test = np.exp(model_log.predict(X_test))
predict_submission = np.exp(model_log.predict(test_data))
VERSION = 4
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test)) * 100:0.2f}%")

Как видим точность возросла до 15%, а что будет на ЛБ?

# GradientBoosting

In [228]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(min_samples_split = 2,
                               learning_rate = 0.03,
                               max_depth = 10,
                               n_estimators = 1000)
gb.fit(X_train, y_train)
predict = gb.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

# Random Forest

In [229]:
rf = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1).fit(X_train, np.log(y_train+1))
predict_submission = np.exp(rf.predict(test_data))
VERSION = 5
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf.predict(X_test)))) * 100:0.2f}%")

# xgboost

In [230]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',
                           colsample_bytree=0.5,
                           learning_rate=0.05,
                           max_depth=12,
                           alpha=1,
                           n_estimators=1000)
xgb_reg.fit(X_train, np.log(y_train))
print(f"Точность модели по метрике MAPE с логтаргетом: {(mape(y_test, np.exp(xgb_reg.predict(X_test))))*100:0.2f}%")

Лучшие показатели у xgboost 13,10, на втором месте CatBoost с логарифмом 13.78 RandomForest 13,79

# Stacking

Cтекинг на базе GradientBoostingRegressor c CatBoostingRegressor и XGBRegressor.

In [231]:
from sklearn.ensemble import StackingRegressor

estimators = [('xgb_reg', xgb.XGBRegressor(objective='reg:squarederror',
                           colsample_bytree=0.5,
                           learning_rate=0.05,
                           max_depth=12,
                           alpha=1,
                           n_estimators=1000)),
              ('model', CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True))]

st_ensemble = StackingRegressor(estimators=estimators,
                                final_estimator=GradientBoostingRegressor(
                                min_samples_split=2,
                                learning_rate=0.03,
                                max_depth=10,
                                n_estimators=1000))


st_ensemble.fit(X_train, np.log(y_train))

predict_ensemble = np.exp(st_ensemble.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_ensemble))*100:0.2f}%")


Cтекинг на базе RandomForestRegressor c CatBoostingRegressor и XGBRegressor.

In [232]:
from sklearn.ensemble import StackingRegressor

estimators = [('xgb_reg', xgb.XGBRegressor(objective='reg:squarederror',
                           colsample_bytree=0.5,
                           learning_rate=0.05,
                           max_depth=12,
                           alpha=1,
                           n_estimators=1000)),
              ('model', CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True))]

st_ensemble_2 = StackingRegressor(estimators=estimators,
                                final_estimator=RandomForestRegressor(
                                random_state = RANDOM_SEED,
                                n_jobs = -1,
                                verbose = 1))


st_ensemble_2.fit(X_train, np.log(y_train))

predict_ensemble_2 = np.exp(st_ensemble_2.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_ensemble_2))*100:0.2f}%")

Как мы видим, несмторя на стекинг данных с использованием моделей с минимальными показателями, лучшей моделью является xgboost с результатом 13.10%

In [233]:
predict_submission = np.exp(xgb_reg.predict(test_data))

# Submission

In [234]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v3.csv', index=False)
sample_submission.head(10)

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML


Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing